# Questão 12

In [1]:
import mysql.connector
import os
from mysql.connector import errorcode
from datetime import date, datetime, timedelta


In [2]:
user_name = os.environ.get('DB_USER')
user_password = os.environ.get('DB_PASS')

In [3]:
cnx = mysql.connector.connect(user=user_name, password=user_password,
                              host='localhost',
                              database='bd')

In [4]:
cnx

In [5]:
TABLES = {}
TABLES['especialista'] = (
    "CREATE TABLE `especialista` ("
    "  `id_esp` int(11) NOT NULL AUTO_INCREMENT,"
    "  `email` varchar(30) NOT NULL,"
    "  `nome` varchar(30) NOT NULL,"
    "  `telefone` int(14) NOT NULL,"
    "  PRIMARY KEY (`id_esp`)"
    ") ENGINE=InnoDB")


In [6]:
TABLES['flor'] = (
    "CREATE TABLE `flor` ("
    "  `id_flor` int(11) NOT NULL,"
    "  `nome` varchar(30) NOT NULL,"
    "  `cod` int(11) NOT NULL,"
    "  PRIMARY KEY (`id_flor`),"
    "  FOREIGN KEY (`id_flor`) "
    "     REFERENCES `especialista` (`id_esp`)"
    ") ENGINE=InnoDB")

In [7]:
cursor = cnx.cursor()

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Criando tabela {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("A tabela já existe.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()

Criando tabela especialista: OK
Criando tabela flor: OK


True

In [8]:
cursor = cnx.cursor()

insert_especialista =("INSERT INTO especialista"
                    "(email, nome, telefone)"
                    "VALUES (%s, %s, %s)")

insert_flor = ("INSERT INTO flor"
                    "(id_flor, nome, cod)"
                    "VALUES (%s, %s, %s)")

data_especialistas = [
    ('Especialista1@gmail.com', 'Especialista 1', 111111111),
    ('Especialista2@gmail.com', 'Especialista 2', 211111111),
    ('Especialista3@gmail.com', 'Especialista 3', 311111111),
]

data_flor = [
    ('flor 1', 1),
    ('flor 2', 2),
    ('flor 3', 3),
    ('flor 5', 4),
    ('flor 4', 5),
]
for x, especilista in enumerate(data_especialistas):
    cursor.execute(insert_especialista, especilista)
    last_id = cursor.lastrowid
    cursor.execute(insert_flor, (last_id,) + data_flor[x])

cnx.commit()

In [9]:
cursor.close

<bound method CMySQLCursor.close of <mysql.connector.cursor_cext.CMySQLCursor object at 0x000002AC43E1A7F0>>

# Questão 13

In [10]:
cursor = cnx.cursor()
sql = "SELECT \
    especialista.nome AS esp, \
    flor.nome AS flor \
    FROM especialista \
    INNER JOIN flor ON especialista.id_esp = flor.id_flor"

cursor.execute(sql)

resultado = cursor.fetchall()

for x in resultado:
    print(x)

('Especialista 1', 'flor 1')
('Especialista 2', 'flor 2')
('Especialista 3', 'flor 3')


# Questão 14

In [11]:
alteracao = ["UPDATE especialista SET nome = %s WHERE email = %s",
            "UPDATE especialista SET telefone = %s WHERE nome = %s"
            ]

valor = [("Especialista 11", "Especialista1@gmail.com"),
            (9999999, "Especialista 3")]

for x, alterar in enumerate(alteracao):
    cursor.execute(alterar, valor[x])

alteracao_flor = "UPDATE flor SET cod = %s WHERE id_flor = %s"
valor = (24, 3)

cursor.execute(alteracao_flor, valor)

cnx.commit()

In [12]:
deletar_flor = "DELETE FROM flor WHERE nome = %s"
deletar_especialista = "DELETE FROM especialista WHERE nome = %s"
valor = ("Flor 2", )
valor_1 = ("Especialista 2", )

cursor.execute(deletar_flor, valor)
cursor.execute(deletar_especialista, valor_1)

cnx.commit()